Arbitrage Analysis & Regression-Based Forecasting


-  Linear Regression | Predict fair value from inputs | Regression, residual analysis
- Arbitrage | Limit Order Book vs Conversion Price | Compare prices if both markets exist
- Fair Value MM | Market make near predicted value | Error vs spread analysis

We can trade macarons in our local market, and we can trade them in the international market. If we want to buy them on the international market, we need to pay the askPrice, import tariff, and transport fees. If we want to sell them on the international market, we need to pay the bidPrice, export tariff, and transport fees. An arbitrage opportunity may be possible if the price difference between the local market and the international market is greater than the sum of the import tariff, export tariff, and transport fees.


In [1]:
import pandas as pd
import os
import glob
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

In [2]:

def load_historical_data(directory_path):
    historical_data = {}
    days = [1, 2, 3]

    if not os.path.isdir(directory_path):
        print(f"Error: Directory not found: {directory_path}")
        return historical_data

    print(f"Loading historical data for Round 4 from: {directory_path}")

    for day in days:
        day_data = {}
        day_str = str(day)

        prices_pattern = os.path.join(directory_path, f'prices_round_4_day_{day_str}.csv')
        trades_pattern = os.path.join(directory_path, f'trades_round_4_day_{day_str}_nn.csv')
        observations_pattern = os.path.join(directory_path, f'observations_round_4_day_{day_str}.csv')

        price_files = glob.glob(prices_pattern)
        trade_files = glob.glob(trades_pattern)
        observations_files = glob.glob(observations_pattern)

        if price_files:
            prices_file_path = price_files[0]
            df_prices = pd.read_csv(prices_file_path, sep=';')
            df_prices['timestamp'] += (day - 1) * 1_000_000
            df_prices['day'] = day
            day_data['prices'] = df_prices

        if trade_files:
            trades_file_path = trade_files[0]
            df_trades = pd.read_csv(trades_file_path, sep=';')
            df_trades['timestamp'] += (day - 1) * 1_000_000
            df_trades['day'] = day
            day_data['trades'] = df_trades

        if observations_files:
            observations_file_path = observations_files[0]
            df_observations = pd.read_csv(observations_file_path, sep=',')
            df_observations['timestamp'] += (day - 1) * 1_000_000
            df_observations['day'] = day
            day_data['observations'] = df_observations

        if day_data:
            historical_data[day] = day_data

    return historical_data

# Load the data
directory_path = 'C:/Users/Admin/projects/prosperity-poj/strategy/round4/resources/round4'
historical_data = load_historical_data(directory_path)

# Combine all data
all_prices = pd.concat([data['prices'] for data in historical_data.values() if 'prices' in data], ignore_index=True)
all_trades = pd.concat([data['trades'] for data in historical_data.values() if 'trades' in data], ignore_index=True)
all_observations = pd.concat([data['observations'] for data in historical_data.values() if 'observations' in data], ignore_index=True)

# Display info
print(all_prices.head())
print(all_trades.head())
print(all_observations.head())

# Initial Data Analysis
print("Price Columns:", all_prices.columns)
print("Trade Columns:", all_trades.columns)
print("Observation Columns:", all_observations.columns)


Loading historical data for Round 4 from: C:/Users/Admin/projects/prosperity-poj/strategy/round4/resources/round4
   day  timestamp                     product  bid_price_1  bid_volume_1  \
0    1          0               VOLCANIC_ROCK      10515.0         143.0   
1    1          0              PICNIC_BASKET1      58705.0          12.0   
2    1          0              PICNIC_BASKET2      30252.0          12.0   
3    1          0  VOLCANIC_ROCK_VOUCHER_9500       1015.0          12.0   
4    1          0  VOLCANIC_ROCK_VOUCHER_9750        766.0          12.0   

   bid_price_2  bid_volume_2  bid_price_3  bid_volume_3  ask_price_1  \
0      10514.0          57.0          NaN           NaN        10517   
1      58704.0          27.0          NaN           NaN        58716   
2      30251.0          27.0          NaN           NaN        30258   
3          NaN           NaN          NaN           NaN         1017   
4          NaN           NaN          NaN           NaN          767 

In [3]:
#Identify arbitrage opportunities


all_prices= all_prices.merge(all_observations, on='timestamp', how='left')


# Filter for MAGNIFICENT_MACARONS (already done in your snippet)
all_prices = all_prices[all_prices['product'] == 'MAGNIFICENT_MACARONS'].copy() # Use .copy() to avoid SettingWithCopyWarning

# Calculate effective conversion prices
# Price to BUY from Chefs (Pristine Cuisine)
all_prices['conversion_buy_price_effective'] = all_prices['askPrice'] + all_prices['transportFees'] + all_prices['importTariff']
# Price to SELL to Chefs (Pristine Cuisine)
all_prices['conversion_sell_price_effective'] = all_prices['bidPrice'] - all_prices['transportFees'] - all_prices['exportTariff']

# --- Arbitrage Scenario 1: Buy on Market, Sell to Chefs ---
# Condition: Market Ask Price < Effective Conversion Sell Price
all_prices['arb1_possible'] = all_prices['ask_price_1'] < all_prices['conversion_sell_price_effective']
all_prices['arb1_profit_per_unit'] = all_prices['conversion_sell_price_effective'] - all_prices['ask_price_1']

# --- Arbitrage Scenario 2: Buy from Chefs, Sell on Market ---
# Condition: Effective Conversion Buy Price < Market Bid Price
all_prices['arb2_possible'] = all_prices['conversion_buy_price_effective'] < all_prices['bid_price_1']
all_prices['arb2_profit_per_unit'] = all_prices['bid_price_1'] - all_prices['conversion_buy_price_effective']

# Filter to show only timestamps where arbitrage is possible
arbitrage_opportunities = all_prices[
    (all_prices['arb1_possible']) | (all_prices['arb2_possible'])
].copy()

# Select and rename columns for clarity
arbitrage_opportunities = arbitrage_opportunities[[
    'timestamp',
    'ask_price_1', 'bid_price_1', # Market Prices
    'conversion_buy_price_effective', 'conversion_sell_price_effective', # Chef Prices
    'arb1_possible', 'arb1_profit_per_unit',
    'arb2_possible', 'arb2_profit_per_unit'
]]

print("\nArbitrage Opportunities Found:")
if arbitrage_opportunities.empty:
    print("No arbitrage opportunities detected in the analyzed data.")
else:
    print(f"Total arbitrage instances found: {len(arbitrage_opportunities)}")
    print("\nArbitrage Scenario 1 (Buy Market, Sell Chef):")
    print(arbitrage_opportunities[arbitrage_opportunities['arb1_possible']][['timestamp', 'ask_price_1', 'conversion_sell_price_effective', 'arb1_profit_per_unit']].head())
    print(f"Count: {arbitrage_opportunities['arb1_possible'].sum()}")

    print("\nArbitrage Scenario 2 (Buy Chef, Sell Market):")
    print(arbitrage_opportunities[arbitrage_opportunities['arb2_possible']][['timestamp', 'conversion_buy_price_effective', 'bid_price_1', 'arb2_profit_per_unit']].head())
    print(f"Count: {arbitrage_opportunities['arb2_possible'].sum()}")

    # Display some overall stats for context
    print("\nOverall Data Sample (showing potential arbitrage columns):")
    print(all_prices[[
        'timestamp','ask_price_1', 'bid_price_1',
        'conversion_buy_price_effective', 'conversion_sell_price_effective',
        'arb1_possible', 'arb2_possible'
        ]].head())
    print(all_prices[[
        'timestamp','ask_price_1', 'bid_price_1',
        'conversion_buy_price_effective', 'conversion_sell_price_effective',
        'arb1_possible', 'arb2_possible'
        ]].tail())




Arbitrage Opportunities Found:
Total arbitrage instances found: 782

Arbitrage Scenario 1 (Buy Market, Sell Chef):
Empty DataFrame
Columns: [timestamp, ask_price_1, conversion_sell_price_effective, arb1_profit_per_unit]
Index: []
Count: 0

Arbitrage Scenario 2 (Buy Chef, Sell Market):
      timestamp  conversion_buy_price_effective  bid_price_1  \
311        2000                           643.5        644.0   
1505      10000                           627.0        628.0   
2795      18600                           615.0        616.0   
3381      22500                           613.0        615.0   
4243      28200                           601.0        602.0   

      arb2_profit_per_unit  
311                    0.5  
1505                   1.0  
2795                   1.0  
3381                   2.0  
4243                   1.0  
Count: 782

Overall Data Sample (showing potential arbitrage columns):
    timestamp  ask_price_1  bid_price_1  conversion_buy_price_effective  \
6       

Analysis of Market Bots